# Flow Preprocessing + Windowing Mechanism Testing and Debugging

In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)

import sys
sys.path.append('..')


%load_ext autoreload
%autoreload 2

In [10]:
# Left out
# 'window_total', 'window_span',
WINDOW_FEATURES = ['flows_total', 'flows_concurrent_max', 'flows_itimes_avg', 'flows_itimes_std', 'flows_itimes_min',
    'flows_itimes_max', 'dur_total', 'dur_avg', 'dur_std', 'pkt_total', 'ppf_avg', 'ppf_std', 'bytes_total', 'bpf_avg',
    'bpf_std', 'bpp_avg', 'bpp_std', 'bps_avg', 'bps_std', 'pps_avg', 'pps_std', 'port_src_uniq_cnt',
    'port_src_entropy', 'proto_all_or', 'flag_syn_ratio', 'flag_ack_ratio', 'flag_fin_ratio', 'ip_dst_uniq',
    'hdr_payload_ratio_avg', 'ttl_std', 'window_active_ratio', 'flows_total_std', 
    'flows_concurrent_max_std', 'flows_itimes_avg_std', 'flows_itimes_std_std', 'flows_itimes_min_std',
    'flows_itimes_max_std', 'dur_total_std', 'dur_avg_std', 'dur_std_std', 'pkt_total_std', 'ppf_avg_std',
    'ppf_std_std', 'bytes_total_std', 'bpf_avg_std', 'bpf_std_std', 'bps_avg_std', 'bps_std_std', 'bpp_avg_std',
    'bpp_std_std', 'port_src_uniq_cnt_std',	'port_src_entropy_std', 'flag_syn_ratio_std', 'flag_ack_ratio_std',
    'flag_fin_ratio_std', 'ip_dst_uniq_std', 'hdr_payload_ratio_avg_std', 'ttl_std_std']

In [2]:
%cd ..

/home/goldy/Documents/vnet/VNET-NAnoDec/src/anomaly_detection/preprocessing


In [3]:
DATAPATH    = '09_sub_preproc_tail.csv'
data = pd.read_csv(DATAPATH)

In [5]:
data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 173 columns):
 #    Column                       Dtype  
---   ------                       -----  
 0    IN_BYTES                     int64  
 1    IN_PKTS                      int64  
 2    OUT_BYTES                    int64  
 3    OUT_PKTS                     int64  
 4    MIN_IP_PKT_LEN               int64  
 5    MAX_IP_PKT_LEN               int64  
 6    MIN_TTL                      int64  
 7    MAX_TTL                      int64  
 8    SRC_FRAGMENTS                int64  
 9    DST_FRAGMENTS                int64  
 10   SRC_TO_DST_AVG_THROUGHPUT    int64  
 11   DST_TO_SRC_AVG_THROUGHPUT    int64  
 12   NUM_PKTS_UP_TO_128_BYTES     int64  
 13   NUM_PKTS_128_TO_256_BYTES    int64  
 14   NUM_PKTS_256_TO_512_BYTES    int64  
 15   NUM_PKTS_512_TO_1024_BYTES   int64  
 16   NUM_PKTS_1024_TO_1514_BYTES  int64  
 17   NUM_PKTS_OVER_1514_BYTES     int64  
 18   LONGEST_FLOW_PKT      

In [6]:
data.describe()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,MIN_IP_PKT_LEN,MAX_IP_PKT_LEN,MIN_TTL,MAX_TTL,SRC_FRAGMENTS,DST_FRAGMENTS,SRC_TO_DST_AVG_THROUGHPUT,DST_TO_SRC_AVG_THROUGHPUT,NUM_PKTS_UP_TO_128_BYTES,NUM_PKTS_128_TO_256_BYTES,NUM_PKTS_256_TO_512_BYTES,NUM_PKTS_512_TO_1024_BYTES,NUM_PKTS_1024_TO_1514_BYTES,NUM_PKTS_OVER_1514_BYTES,LONGEST_FLOW_PKT,SHORTEST_FLOW_PKT,RETRANSMITTED_IN_PKTS,RETRANSMITTED_OUT_PKTS,OOORDER_IN_PKTS,OOORDER_OUT_PKTS,DURATION_IN,DURATION_OUT,TCP_WIN_MIN_IN,TCP_WIN_MAX_IN,TCP_WIN_MSS_IN,TCP_WIN_SCALE_IN,TCP_WIN_MIN_OUT,TCP_WIN_MAX_OUT,TCP_WIN_MSS_OUT,TCP_WIN_SCALE_OUT,SRC_TO_DST_IAT_MIN,SRC_TO_DST_IAT_MAX,SRC_TO_DST_IAT_AVG,SRC_TO_DST_IAT_STDDEV,DST_TO_SRC_IAT_MIN,DST_TO_SRC_IAT_MAX,DST_TO_SRC_IAT_AVG,DST_TO_SRC_IAT_STDDEV,flows_total,flows_concurrent_max,flows_itimes_avg,flows_itimes_std,flows_itimes_min,flows_itimes_max,dur_total,dur_avg,dur_std,pkt_total,ppf_avg,ppf_std,bytes_total,bpf_avg,bpf_std,bpp_avg,bpp_std,bps_avg,bps_std,pps_avg,pps_std,port_src_uniq_cnt,port_src_entropy,proto_all_or,flag_syn_ratio,flag_ack_ratio,flag_fin_ratio,ip_dst_uniq,hdr_payload_ratio_avg,ttl_std,window_total,window_span,window_active_ratio,flows_total_std,flows_concurrent_max_std,flows_itimes_avg_std,flows_itimes_std_std,flows_itimes_min_std,flows_itimes_max_std,dur_total_std,dur_avg_std,dur_std_std,pkt_total_std,ppf_avg_std,ppf_std_std,bytes_total_std,bpf_avg_std,bpf_std_std,bps_avg_std,bps_std_std,bpp_avg_std,bpp_std_std,port_src_uniq_cnt_std,port_src_entropy_std,flag_syn_ratio_std,flag_ack_ratio_std,flag_fin_ratio_std,ip_dst_uniq_std,hdr_payload_ratio_avg_std,ttl_std_std,IN_BPS,IN_BPP,IN_PPS,OUT_BPS,OUT_BPP,OUT_PPS,FLAG_FIN,FLAG_SYN,FLAG_RST,FLAG_PSH,FLAG_ACK,FLAG_URG,FLAG_ECE,FLAG_CWR,CLIENT_FLAG_FIN,CLIENT_FLAG_SYN,CLIENT_FLAG_RST,CLIENT_FLAG_PSH,CLIENT_FLAG_ACK,CLIENT_FLAG_URG,CLIENT_FLAG_ECE,CLIENT_FLAG_CWR,SERVER_FLAG_FIN,SERVER_FLAG_SYN,SERVER_FLAG_RST,SERVER_FLAG_PSH,SERVER_FLAG_ACK,SERVER_FLAG_URG,SERVER_FLAG_ECE,SERVER_FLAG_CWR,PROTOCOL_ICMP,PROTOCOL_TCP,PROTOCOL_UDP,PROTOCOL_GRE,PROTOCOL_ESP,PROTOCOL_ICMP6,L4_SRC_PORT_OTHER,L4_SRC_PORT_NOPORT,L4_SRC_PORT_WEB_NOENC,L4_SRC_PORT_TLS,L4_SRC_PORT_DNS,L4_SRC_PORT_EMAIL,L4_SRC_PORT_VPN,L4_SRC_PORT_DATA,L4_SRC_PORT_SHELL,L4_SRC_PORT_PLAYSTATION,L4_SRC_PORT_CHAT,L4_SRC_PORT_QUERY,L4_SRC_PORT_DYNAMIC,L4_DST_PORT_OTHER,L4_DST_PORT_NOPORT,L4_DST_PORT_WEB_NOENC,L4_DST_PORT_TLS,L4_DST_PORT_DNS,L4_DST_PORT_EMAIL,L4_DST_PORT_VPN,L4_DST_PORT_DATA,L4_DST_PORT_SHELL,L4_DST_PORT_PLAYSTATION,L4_DST_PORT_CHAT,L4_DST_PORT_QUERY,L4_DST_PORT_DYNAMIC,ICMP_TYPE_OTHER,ICMP_TYPE_ECHO_REPLY,ICMP_TYPE_DEST_UNREACHABLE,ICMP_TYPE_REDIRECT,ICMP_TYPE_ECHO_REQUEST,ICMP_TYPE_TIME_EXCEEDED
count,5.000000e+05,5.000000e+05,5.000000e+05,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,5.000000e+05,5.000000e+05,500000.000000,500000.000000,500000.000000,500000.000000,5.000000e+05,500000.00000,500000.000000,500000.000000,500000.000000,500000.00000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,5.000000e+05,500000.000000,5.000000e+05,500000.000000,500000.000000,5.000000e+05,5.000000e+05,5.000000e+05,5.000000e+05,500000.000000,500000.000000,5.000000e+05,5.000000e+05,500000.000000,5.000000e+05,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,5.000000e+05,500000.000000,5.000000e+05,500000.000000,500000.000000,5.000000e+05,5.000000e+05,5.000000e+05,5.000000e+05,5.000000e+05,5.000000e+05,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500

In [18]:
# Determine the number of all 0s rows
((data[WINDOW_FEATURES] == 0).sum(axis=1) == len(WINDOW_FEATURES)).sum()

136924

In [19]:
# And the ratio is
((data[WINDOW_FEATURES] == 0).sum(axis=1) == len(WINDOW_FEATURES)).sum() / len(data)

0.273848